# NER TRAINING WITH ONLY PERSON ENTITY DATASET

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

# Open the JSON file
with open("/content/drive/MyDrive/NER/merged_data.json", "r") as file:
    # Load the JSON data
    dataset = json.load(file)

len(dataset)

2895

In [ ]:
import random
from spacy.util import minibatch, compounding


# Set the random seed for reproducibility
random.seed(42)

# Shuffle the dataset
random.shuffle(dataset)

# Define the split ratios
train_ratio = 0.6  # 80% for training
val_ratio = 0.2  # 10% for validation
test_ratio = 0.2  # 10% for testing

# Calculate the sizes of each split
total_size = len(dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)

# Split the dataset into train, validation, and test sets
train_data = dataset[:train_size]
val_data = dataset[train_size:train_size + val_size]
test_data = dataset[train_size + val_size:]

# Print the sizes of each split
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


Training set size: 1737
Validation set size: 579
Test set size: 579


In [ ]:
!mkdir data

In [ ]:
with open("data/train.json", 'w') as fp:
        json.dump(train_data, fp, indent=2, ensure_ascii=False)


with open("data/test.json", 'w') as fp:
        json.dump(test_data, fp, indent=2, ensure_ascii=False)


with open("data/val.json", 'w') as fp:
        json.dump(val_data, fp, indent=2, ensure_ascii=False)


In [ ]:
!python -m spacy convert data/train.json ./data
!python -m spacy convert data/test.json ./data
!python -m spacy convert data/val.json ./data

2023-06-26 17:31:02.286054: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (1737 documents): data/train.spacy
2023-06-26 17:31:11.118503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (579 documents): data/test.spacy
2023-06-26 17:31:19.014766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (579 documents): data/val.spacy


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-06-26 17:31:36.341123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train data/train.spacy --paths.dev data/val.spacy --gpu-id 0

2023-06-26 17:31:47.717097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-26 17:31:54,461] [INFO] Set up nlp object from config
[2023-06-26 17:31:54,476] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-26 17:31:54,480] [INFO] Created vocabulary
[2023-06-26 17:31:54,480] [INFO] Finished initializing nlp object
[2023-06-26 17:32:13,475] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     40.17    0.00    0.00    0.00    0.00
  0     200      

In [ ]:
import spacy

nlp = spacy.load("/content/output/model-best")

text_list = [
    "বিএনপি নেতাদের সবচেয়ে বড় দুর্বলতা তাঁরা রাজনৈতিক কর্মী আর সন্ত্রাসী গুলিয়ে ফেলেছেন বলে মন্তব্য করেছেন তথ্যমন্ত্রী হাছান মাহমুদ",
    "আব্দুর রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম",
    "পুতিনের দুর্বলতা ফাঁস করে দিয়েছে ভাগনার",
    "এবার বাবা শাহরুখের ছবিতে অভিনয় করবেন সুহানা",
    "বিনা প্রতিদ্বন্দ্বিতায় বান্দরবান পৌরসভার মেয়র হলেন আওয়ামী লীগের শামসুল",
    "রাত সাড়ে নয়টার দিকে রিটার্নিং কর্মকর্তা ফয়সল কাদের বেসরকারিভাবে আওয়ামী লীগের প্রার্থী মো. আনোয়ারুজ্জামান চৌধুরীকে বিজয়ী ঘোষণা করেন",
    " যাঁরা জায়েদ খানের পারফরম্যান্স দেখতে চান, তাঁরা আগামী দিনে ওয়াশিংটনে গেলে দেখতে পাবেন।"

]
for text in text_list:
    doc = nlp(text)

    print(f"Input: {text}")
    for entity in doc.ents:
        print(f"Entity: {entity.text}, Label: {entity.label_}")
    print("---")



Input: বিএনপি নেতাদের সবচেয়ে বড় দুর্বলতা তাঁরা রাজনৈতিক কর্মী আর সন্ত্রাসী গুলিয়ে ফেলেছেন বলে মন্তব্য করেছেন তথ্যমন্ত্রী হাছান মাহমুদ
Entity: হাছান মাহমুদ, Label: PER
---
Input: আব্দুর রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম
Entity: আব্দুর রহিম, Label: PER
---
Input: পুতিনের দুর্বলতা ফাঁস করে দিয়েছে ভাগনার
---
Input: এবার বাবা শাহরুখের ছবিতে অভিনয় করবেন সুহানা
Entity: শাহরুখের, Label: PER
---
Input: বিনা প্রতিদ্বন্দ্বিতায় বান্দরবান পৌরসভার মেয়র হলেন আওয়ামী লীগের শামসুল
---
Input: রাত সাড়ে নয়টার দিকে রিটার্নিং কর্মকর্তা ফয়সল কাদের বেসরকারিভাবে আওয়ামী লীগের প্রার্থী মো. আনোয়ারুজ্জামান চৌধুরীকে বিজয়ী ঘোষণা করেন
Entity: ফয়সল কাদের, Label: PER
Entity: মো. আনোয়ারুজ্জামান চৌধুরীকে, Label: PER
---
Input:  যাঁরা জায়েদ খানের পারফরম্যান্স দেখতে চান, তাঁরা আগামী দিনে ওয়াশিংটনে গেলে দেখতে পাবেন।
Entity: জায়েদ খানের, Label: PER
---


In [ ]:
import os
# os.mkdir('result')
!python -m spacy evaluate output//model-best data/test.spacy

2023-06-26 17:48:26.197811: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     -    
NER P   79.13
NER R   63.42
NER F   70.41
SPEED   21952


=============================== NER (per type) ===============================

          P       R       F
PER   79.13   63.42   70.41



In [ ]:
!python -m spacy evaluate output//model-best data/val.spacy

2023-06-26 17:49:22.525659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     -    
NER P   79.20
NER R   63.82
NER F   70.68
SPEED   22291


=============================== NER (per type) ===============================

          P       R       F
PER   79.20   63.82   70.68



# All Entity Considered

In [ ]:
import json

# Open the JSON file
with open("/content/drive/MyDrive/NER/all_entity_merged_data.json", "r") as file:
    # Load the JSON data
    dataset = json.load(file)

len(dataset)

6685

In [ ]:
import random
from spacy.util import minibatch, compounding


# Set the random seed for reproducibility
random.seed(42)

# Shuffle the dataset
random.shuffle(dataset)

# Define the split ratios
train_ratio = 0.6  # 80% for training
val_ratio = 0.2  # 10% for validation
test_ratio = 0.2  # 10% for testing

# Calculate the sizes of each split
total_size = len(dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)

# Split the dataset into train, validation, and test sets
train_data = dataset[:train_size]
val_data = dataset[train_size:train_size + val_size]
test_data = dataset[train_size + val_size:]

# Print the sizes of each split
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


Training set size: 4011
Validation set size: 1337
Test set size: 1337


In [ ]:
import os
!mkdir "all_data"

In [ ]:
with open("all_data/train.json", 'w') as fp:
        json.dump(train_data, fp, indent=2, ensure_ascii=False)


with open("all_data/test.json", 'w') as fp:
        json.dump(test_data, fp, indent=2, ensure_ascii=False)


with open("all_data/val.json", 'w') as fp:
        json.dump(val_data, fp, indent=2, ensure_ascii=False)


In [ ]:
!python -m spacy convert all_data/train.json ./all_data
!python -m spacy convert all_data/test.json ./all_data
!python -m spacy convert all_data/val.json ./all_data

2023-06-26 17:53:43.243253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (4011 documents): all_data/train.spacy
2023-06-26 17:53:53.005721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (1337 documents): all_data/test.spacy
2023-06-26 17:54:01.822181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (1337 documents): all_data/val.spacy


In [ ]:
!python -m spacy train config.cfg --output ./all_output --paths.train all_data/train.spacy --paths.dev all_data/val.spacy --gpu-id 0

2023-06-26 17:54:48.795012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: all_output
ℹ Saving to output directory: all_output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-26 17:54:52,283] [INFO] Set up nlp object from config
[2023-06-26 17:54:52,300] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-06-26 17:54:52,303] [INFO] Created vocabulary
[2023-06-26 17:54:52,303] [INFO] Finished initializing nlp object
[2023-06-26 17:55:05,333] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     49.43    0.00    0.00    0.00    0.00
  0     2

In [ ]:
import spacy

nlp = spacy.load("/content/all_output/model-best")

text_list = [
    "বিএনপি নেতাদের সবচেয়ে বড় দুর্বলতা তাঁরা রাজনৈতিক কর্মী আর সন্ত্রাসী গুলিয়ে ফেলেছেন বলে মন্তব্য করেছেন তথ্যমন্ত্রী হাছান মাহমুদ",
    "আব্দুর রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম",
    "পুতিনের দুর্বলতা ফাঁস করে দিয়েছে ভাগনার",
    "এবার বাবা শাহরুখের ছবিতে অভিনয় করবেন সুহানা",
    "বিনা প্রতিদ্বন্দ্বিতায় বান্দরবান পৌরসভার মেয়র হলেন আওয়ামী লীগের শামসুল",
    "রাত সাড়ে নয়টার দিকে রিটার্নিং কর্মকর্তা ফয়সল কাদের বেসরকারিভাবে আওয়ামী লীগের প্রার্থী মো. আনোয়ারুজ্জামান চৌধুরীকে বিজয়ী ঘোষণা করেন",
    " যাঁরা জায়েদ খানের পারফরম্যান্স দেখতে চান, তাঁরা আগামী দিনে ওয়াশিংটনে গেলে দেখতে পাবেন।"

]
for text in text_list:
    doc = nlp(text)

    print(f"Input: {text}")
    for entity in doc.ents:
        print(f"Entity: {entity.text}, Label: {entity.label_}")
    print("---")



Input: বিএনপি নেতাদের সবচেয়ে বড় দুর্বলতা তাঁরা রাজনৈতিক কর্মী আর সন্ত্রাসী গুলিয়ে ফেলেছেন বলে মন্তব্য করেছেন তথ্যমন্ত্রী হাছান মাহমুদ
Entity: বিএনপি, Label: ORG
Entity: হাছান মাহমুদ, Label: PER
---
Input: আব্দুর রহিম নামের কাস্টমারকে একশ টাকা বাকি দিলাম
Entity: আব্দুর রহিম, Label: PER
---
Input: পুতিনের দুর্বলতা ফাঁস করে দিয়েছে ভাগনার
Entity: পুতিনের, Label: ORG
Entity: ভাগনার, Label: LOC
---
Input: এবার বাবা শাহরুখের ছবিতে অভিনয় করবেন সুহানা
Entity: শাহরুখের, Label: PER
---
Input: বিনা প্রতিদ্বন্দ্বিতায় বান্দরবান পৌরসভার মেয়র হলেন আওয়ামী লীগের শামসুল
Entity: আওয়ামী লীগের, Label: ORG
---
Input: রাত সাড়ে নয়টার দিকে রিটার্নিং কর্মকর্তা ফয়সল কাদের বেসরকারিভাবে আওয়ামী লীগের প্রার্থী মো. আনোয়ারুজ্জামান চৌধুরীকে বিজয়ী ঘোষণা করেন
Entity: রাত, Label: M
Entity: নয়টার, Label: M
Entity: ফয়সল কাদের, Label: PER
Entity: আওয়ামী লীগের, Label: ORG
Entity: মো. আনোয়ারুজ্জামান চৌধুরীকে, Label: PER
---
Input:  যাঁরা জায়েদ খানের পারফরম্যান্স দেখতে চান, তাঁরা আগামী দিনে ওয়াশিংটনে গেলে দেখতে পাবেন।
Entity: 

In [ ]:
!python -m spacy evaluate all_output//model-best all_data/val.spacy

2023-06-26 18:07:54.013097: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     -    
NER P   70.95
NER R   60.78
NER F   65.47
SPEED   23055


=============================== NER (per type) ===============================

           P       R       F
M      81.22   79.18   80.19
PER    64.72   59.62   62.07
LOC    70.89   61.31   65.75
ORG    72.23   53.85   61.70
DATE   84.27   75.00   79.37
GPE    62.57   47.95   54.29
LAW     0.00    0.00    0.00



In [ ]:
import os
# os.mkdir('all_result')
!python -m spacy evaluate all_output//model-best all_data/test.spacy

2023-06-26 18:08:04.154752: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     -    
NER P   70.41
NER R   60.49
NER F   65.07
SPEED   23460


=============================== NER (per type) ===============================

           P       R       F
M      82.31   78.05   80.13
ORG    66.40   52.16   58.42
GPE    57.81   45.68   51.03
LOC    67.81   55.87   61.26
PER    70.47   62.82   66.43
DATE   68.09   64.00   65.98
LAW    75.00   42.86   54.55

